In [37]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

print('This is loaded')

This is loaded


In [38]:
base = pd.read_csv('rc15csv/pai2015before.csv', thousands=",", index_col='schid')
print('1 loaded')

1 loaded


In [39]:
base.head(3)

,schid14,schid15,value,facilityname,selective,districtname,distid,distidalt,county,districttype,...,me2015distAPP,me2015distMET,me2015distEXC,me2015stateDNY,me2015statePART,me2015stateAPP,me2015stateMET,me2015stateEXC,me2015schoolELEM,me2015schoolHS
schid,,,,,,,,,,,,,,,,,,,,,
010010010260001,010010010260001,010010010260001,"Seymour High School, Payson",Seymour High School,--,Payson CUSD 1,010010010260000,010010010260000,Adams,UNIT,...,30.2,30.2,2.9,14.4,24.2,28.4,28.6,4.4,NaN,25.6
010010010262002,010010010262002,010010010262002,"Seymour Elementary School, Payson",Seymour Elementary School,--,Payson CUSD 1,010010010260000,010010010260000,Adams,UNIT,...,30.2,30.2,2.9,14.4,24.2,28.4,28.6,4.4,36.1,NaN
010010020260001,010010020260001,010010020260001,"Liberty High School, Liberty",Liberty High School,--,Liberty CUSD 2,010010020260000,010010020260000,Adams,UNIT,...,37.6,28.6,2.9,14.4,24.2,28.4,28.6,4.4,NaN,30.9


In [40]:
base['zelemscore2015me'] = 500.0
base['avgelem2015me'] = ''
base['stdvelem2015me'] = ''
base['zelemscore2015meapp'] = 500.0
base['avgelem2015meapp'] = ''
base['stdvelem2015meapp'] = ''
base['avgELEMz'] = 500.0
base['zhsscore2015me'] = 500.0
base['avghs2015me'] = ''
base['stdvhs2015me'] = ''
base['zhsscore2015meapp'] = 500.0
base['avghs2015meapp'] = ''
base['stdvhs2015meapp'] = ''
base['avgHSz'] = 500.0

In [41]:
base.shape

(4094, 255)

In [42]:
elemval = ['CHARTER NET','CHARTER NET+']
elembase = base[~base.schooltype.isin(elemval)]

In [43]:
elembase['MEappelem'] = elembase['me2015schoolELEM'] + elembase['me2015schAPP']
elembase.shape

(4084, 256)

In [44]:
#figure elem schools

lowincrange = {0: 10, 10: 20, 20: 30, 30: 40, 40: 50, 50: 60, 60: 70, 70: 80, 80: 90, 90: 101}

for key in lowincrange:
    dfselect = elembase[ elembase["me2015schoolELEM"].notnull() & ( elembase['lowinc2015'] >= key ) & ( elembase['lowinc2015'] < lowincrange[key] ) ]

    meavg = round(dfselect["me2015schoolELEM"].mean(),1)
    mestd = round(dfselect["me2015schoolELEM"].std(ddof=1),2)
    dfselect.loc[:, 'avgelem2015me'] = meavg
    dfselect.loc[:, 'stdvelem2015me'] = mestd    
    dfselect.loc[:,'zelemscore2015me'] = ( ( dfselect['me2015schoolELEM'] - meavg) / mestd ).round(2)
    ssize = len(dfselect)
    sterror = round((mestd / np.sqrt(ssize)) * 1.98,2)
    low95 = round(meavg - sterror,2)
    high95 = round(meavg + sterror,2)

    meAPPavg = round(dfselect["MEappelem"].mean(),1)
    meAPPstd = round(dfselect["MEappelem"].std(ddof=1),2)
    dfselect.loc[:, 'avgelem2015meapp'] = meavg
    dfselect.loc[:, 'stdvelem2015meapp'] = mestd    
    dfselect.loc[:,'zelemscore2015meapp'] = ( ( dfselect['MEappelem'] - meAPPavg) / meAPPstd ).round(2)
    sterrorAPP = round((meAPPstd / np.sqrt(ssize)) * 1.98,2)
    low95APP = round(meAPPavg - sterror,2)
    high95APP = round(meAPPavg + sterror,2)

    dfselect.loc[:,'avgELEMz'] = ( ( dfselect['zelemscore2015meapp'] + dfselect['zelemscore2015me'] ) /2).round(2)

    print("Range ME | ME | Sample | StDev | Error | 95% Low | 95% High")
    print( key,"to",lowincrange[key]," | ",meavg," | ",ssize," | ",mestd," | ",sterror," | ",low95," | ",high95 )
    print("Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High")
    print( key,"to",lowincrange[key]," | ",meAPPavg," | ",ssize," | ",meAPPstd," | ",sterrorAPP," | ",low95APP," | ",high95APP )

    base.update(dfselect)
    
print('done')

Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
0 to 10  |  63.8  |  172  |  10.67  |  1.61  |  62.19  |  65.41
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
0 to 10  |  87.6  |  172  |  5.88  |  0.89  |  85.99  |  89.21
Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
80 to 90  |  18.7  |  217  |  8.92  |  1.2  |  17.5  |  19.9
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
80 to 90  |  46.2  |  217  |  12.32  |  1.66  |  45.0  |  47.4
Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
50 to 60  |  29.8  |  347  |  9.61  |  1.02  |  28.78  |  30.82
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
50 to 60  |  61.3  |  347  |  10.57  |  1.12  |  60.28  |  62.32
Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
20 to 30  |  48.6  |  281  |  12.72  |  1.5  |  47.1  |  50.1
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
20 to 30  |  77.2  |  281  |  8.91  |  1.05  |

In [45]:
base.shape

(4094, 255)

In [46]:
base['MEappHS'] = base['me2015schoolHS'] + base['me2015schAPP']
base.shape

(4094, 256)

In [47]:
#figure high schools

lowincrange = {0: 12.5, 12.5: 25, 25: 37.5, 37.5: 50, 50: 62.5, 62.5: 75, 75: 87.5, 87.5: 101}

for key in lowincrange:
    dfselect = base[ base["me2015schoolHS"].notnull() & ( base['lowinc2015'] >= key ) & ( base['lowinc2015'] < lowincrange[key] ) ]

    meavg = round(dfselect["me2015schoolHS"].mean(),1)
    mestd = round(dfselect["me2015schoolHS"].std(ddof=1),2)
    dfselect.loc[:, 'avghs2015me'] = meavg
    dfselect.loc[:, 'stdvhs2015me'] = mestd
    dfselect.loc[:,'zhsscore2015me'] = ( ( dfselect['me2015schoolHS'] - meavg) / mestd ).round(2)
    ssize = len(dfselect)
    sterror = round((mestd / np.sqrt(ssize)) * 1.98,2)
    low95 = round(meavg - sterror,2)
    high95 = round(meavg + sterror,2)

    meAPPavg = round(dfselect["MEappHS"].mean(),1)
    meAPPstd = round(dfselect["MEappHS"].std(ddof=1),2)
    dfselect.loc[:, 'avghs2015meapp'] = meavg
    dfselect.loc[:, 'stdvhs2015meapp'] = mestd    
    dfselect.loc[:,'zhsscore2015meapp'] = ( ( dfselect['MEappHS'] - meAPPavg) / meAPPstd ).round(2)
    sterrorAPP = round((meAPPstd / np.sqrt(ssize)) * 1.98,2)
    low95APP = round(meAPPavg - sterror,2)
    high95APP = round(meAPPavg + sterror,2)

    dfselect.loc[:,'avgHSz'] = ( ( dfselect['zhsscore2015meapp'] + dfselect['zhsscore2015me'] ) /2).round(2)
    
    print("Range ME | ME | Sample | StDev | Error | 95% Low | 95% High")
    print( key,"to",lowincrange[key]," | ",meavg," | ",ssize," | ",mestd," | ",sterror," | ",low95," | ",high95 )
    print("Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High")
    print( key,"to",lowincrange[key]," | ",meAPPavg," | ",ssize," | ",meAPPstd," | ",sterrorAPP," | ",low95APP," | ",high95APP )

    base.update(dfselect)
    
print('done')

Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
0 to 12.5  |  40.6  |  30  |  16.92  |  6.12  |  34.48  |  46.72
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
0 to 12.5  |  68.7  |  30  |  18.56  |  6.71  |  62.58  |  74.82
Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
50 to 62.5  |  24.3  |  87  |  12.1  |  2.57  |  21.73  |  26.87
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
50 to 62.5  |  52.0  |  87  |  14.67  |  3.11  |  49.43  |  54.57
Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
37.5 to 50  |  27.1  |  168  |  13.86  |  2.12  |  24.98  |  29.22
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
37.5 to 50  |  54.8  |  168  |  15.27  |  2.33  |  52.68  |  56.92
Range ME | ME | Sample | StDev | Error | 95% Low | 95% High
87.5 to 101  |  7.8  |  88  |  8.57  |  1.81  |  5.99  |  9.61
Range ME+APP | ME+APP | Sample | StDev | Error | 95% Low | 95% High
87.5 to 101  |  26.3  |  88  |

In [48]:
base.head(3)

,schid14,schid15,value,facilityname,selective,districtname,distid,distidalt,county,districttype,...,stdvelem2015meapp,avgELEMz,zhsscore2015me,avghs2015me,stdvhs2015me,zhsscore2015meapp,avghs2015meapp,stdvhs2015meapp,avgHSz,MEappHS
schid,,,,,,,,,,,,,,,,,,,,,
010010010260001,010010010260001,010010010260001,"Seymour High School, Payson",Seymour High School,--,Payson CUSD 1,010010010260000,010010010260000,Adams,UNIT,...,,500.00,-0.11,27.1,13.86,0.03,27.1,13.86,-0.04,55.2
010010010262002,010010010262002,010010010262002,"Seymour Elementary School, Payson",Seymour Elementary School,--,Payson CUSD 1,010010010260000,010010010260000,Adams,UNIT,...,9.61,0.57,500.00,,,500.00,,,500.00,NaN
010010020260001,010010020260001,010010020260001,"Liberty High School, Liberty",Liberty High School,--,Liberty CUSD 2,010010020260000,010010020260000,Adams,UNIT,...,,500.00,0.01,30.7,13.64,0.80,30.7,13.64,0.40,71.5


In [49]:
base.loc[base['zelemscore2015me'] == 500.0, 'zelemscore2015me'] = ''
base.loc[base['zelemscore2015meapp'] == 500.0, 'zelemscore2015meapp'] = ''
base.loc[base['avgELEMz'] == 500.0, 'avgELEMz'] = ''
base.loc[base['zhsscore2015me'] == 500.0, 'zhsscore2015me'] = ''
base.loc[base['zhsscore2015meapp'] == 500.0, 'zhsscore2015meapp'] = ''
base.loc[base['avgHSz'] == 500.0, 'avgHSz'] = ''
base.head(3)

,schid14,schid15,value,facilityname,selective,districtname,distid,distidalt,county,districttype,...,stdvelem2015meapp,avgELEMz,zhsscore2015me,avghs2015me,stdvhs2015me,zhsscore2015meapp,avghs2015meapp,stdvhs2015meapp,avgHSz,MEappHS
schid,,,,,,,,,,,,,,,,,,,,,
010010010260001,010010010260001,010010010260001,"Seymour High School, Payson",Seymour High School,--,Payson CUSD 1,010010010260000,010010010260000,Adams,UNIT,...,,,-0.11,27.1,13.86,0.03,27.1,13.86,-0.04,55.2
010010010262002,010010010262002,010010010262002,"Seymour Elementary School, Payson",Seymour Elementary School,--,Payson CUSD 1,010010010260000,010010010260000,Adams,UNIT,...,9.61,0.57,,,,,,,,NaN
010010020260001,010010020260001,010010020260001,"Liberty High School, Liberty",Liberty High School,--,Liberty CUSD 2,010010020260000,010010020260000,Adams,UNIT,...,,,0.01,30.7,13.64,0.8,30.7,13.64,0.4,71.5


In [50]:
base = base.drop('MEappHS', 1)
base.shape

(4094, 255)

In [51]:
#for me to fiddle with
base.to_csv("rc15csv/pai2015done.csv")